# Value Iteration



### Imports

In [ ]:
import numpy as np
import pandas as pd
import time
from IPython.display import display, clear_output

### Environment Definition and Setup



In [ ]:
ROWS = 10
COLS = 10
STATES = [(row, col) for row in range(ROWS) for col in range(COLS)]

"""
(0, 0), (0, 1), (0, 2),
(1, 0), (1, 1), (1, 2),
(2, 0), (2, 1), (2, 2)
"""

GOAL = (5, 4)
GOAL_REWARD = 100
PITS = {(3, 2), (4,8), (9, 3), (7, 6), (6, 1), (1, 5), (4, 4), (5, 6)}
PIT_REWARD = -50
TERMINAL_STATES = {GOAL}|PITS

STEP_REWARD = -1

# BONUSES = {(5, 5)}
# BONUS_RWARD = 10

ACTIONS = {
    'UP': (-1, 0),
    'DOWN': (1, 0),
    'LEFT': (0, -1),
    'RIGHT': (0, 1)
}

# R(x,x,s') - the immidiate reward for moving into s'=next_state
def get_reward(next_state):
    reward = 0
    if next_state == GOAL:
        reward += GOAL_REWARD
    if next_state in PITS:
        reward += PIT_REWARD
    # if next_state in BONUSES:
    #     reward += BONUS_REWARD
    return reward + STEP_REWARD        # 1 point penalty for all moves


GAMMA = 0.9  # Discount factor
CONVERGENCE_THRESHOLD = 0.1  # When to stop iterating


### Helper Functions

In [ ]:
ACTION_ARROWS = {'UP': '↑', 'DOWN': '↓', 'LEFT': '←', 'RIGHT': '→'}  # For drawing the optimal policy grid later on

def get_next_state(state, action):  # Returns (s', R(s,a,s'))
    # If state is terminal it stays terminal
    if state in TERMINAL_STATES:
        return state, 0

    next_state = (state[0]+action[0], state[1]+action[1])
    # If new row or col is out of bounds stay in the same state
    if not (0 <= next_state[0] < ROWS and 0 <= next_state[1] < COLS):
        next_state = state

    return next_state, get_reward(next_state)

def format_table(data, title=""):  # Prints tables nicely using Pandas
    df = pd.DataFrame(data)
    if title:
        print(title)
    display(df.style.format(lambda x: f"{x:,.2f}" if isinstance(x, float) else x))

#### initial Board

In [ ]:
grid = [[get_reward((row, col)) for col in range(COLS)] for row in range(ROWS)]
grid_df = pd.DataFrame(np.full((ROWS, COLS), '', dtype=object))
for state in STATES:
  if state in TERMINAL_STATES:
    grid_df.iloc[state] = f"[{get_reward(state)}]"
  else:
    grid_df.iloc[state] = f"({get_reward(state)})"

format_table(grid_df, "Step-To Rewards Per State\n() = Normal State, [] = Terminal State")

Step-To Rewards Per State
() = Normal State, [] = Terminal State


,0,1,2,3,4,5,6,7,8,9
0,(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1)
1,(-1),(-1),(-1),(-1),(-1),[-51],(-1),(-1),(-1),(-1)
2,(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1)
3,(-1),(-1),[-51],(-1),(-1),(-1),(-1),(-1),(-1),(-1)
4,(-1),(-1),(-1),(-1),[-51],(-1),(-1),(-1),[-51],(-1)
5,(-1),(-1),(-1),(-1),[99],(-1),[-51],(-1),(-1),(-1)
6,(-1),[-51],(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1)
7,(-1),(-1),(-1),(-1),(-1),(-1),[-51],(-1),(-1),(-1)
8,(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1),(-1)
9,(-1),(-1),(-1),[-51],(-1),(-1),(-1),(-1),(-1),(-1)


### Value Iteration Algorithm

In [ ]:
def value_iteration(print_all_iterations=False, delay=1):
  # Initialize V(s) = 0 for all states
  V = np.zeros((ROWS, COLS))

  iteration = 0
  while True:
    iteration += 1
    V_new = V.copy()  # New value table for this iteration
    max_delta = 0  # Max change in this iteration

    # Display the current iteration table
    if not print_all_iterations:
      clear_output(wait=True)
    format_table(V, f"--- Iteration {iteration} ---")
    if not print_all_iterations:
      time.sleep(delay) # Pause to make it viewable

    # Loop through all states
    for state in STATES:
      # If it's a terminal state, its value is 0 (no future)
      if state in TERMINAL_STATES:
        V_new[state[0], state[1]] = 0
        continue

      # Calculate the value for each possible action - R(s, a, s') + γ * V_k(s') for each s'
      action_values = []
      for action in ACTIONS.values():
        next_state, reward = get_next_state(state, action)

        # Bellman update equation: R(s, a, s') + γ * V_k(s')
        v = reward + GAMMA * V[next_state[0], next_state[1]]
        action_values.append(v)

      # V_k+1(s) = max_a [ R(s, a, s') + γ * V_k(s') ]
      best_value = max(action_values)
      V_new[state[0], state[1]] = best_value

      # Update the max change this iteration (delta). Used for convergence checking.
      state_delta = abs(V_new[state[0], state[1]] - V[state[0], state[1]])
      max_delta = max(max_delta, state_delta)

    # Finished looping over states
    # Update the value table for the next iteration
    V = V_new

    # Check for convergence
    if max_delta < CONVERGENCE_THRESHOLD:
      if not print_all_iterations:
        clear_output(wait=True)
      print(f"Convergence reached at iteration {iteration}!")
      break

  return V

### Policy Extraction

In [ ]:
def extract_policy(V):
  policy = np.full((ROWS, COLS), '', dtype=object)

  for state in STATES:
    # Handle terminal states
    if state == GOAL:
        policy[state[0], state[1]] = "👑"
        continue
    if state in PITS:
        policy[state[0], state[1]] = "💣"
        continue

    # Find the best action
    best_action = None
    best_value = -np.inf

    # Iterate over all possible actions a
    for action_name, action_delta in ACTIONS.items():
        next_state, reward = get_next_state(state, action_delta)  # s', R(s, a, s')

        # v = R(s, a, s') + γ * V*(s') for CURRENT ACTION a
        v = reward + GAMMA * V[next_state[0], next_state[1]]

        if v > best_value:
            best_value = v
            best_action = ACTION_ARROWS[action_name]

    policy[state[0], state[1]] = best_action

  return policy

### Execution

In [ ]:
if __name__ == "__main__":
    # Note: This check won't run in a Colab cell,
    # but the code will run sequentially.

    # Run Value Iteration
    optimal_V = value_iteration(False, 1)

    # Display the final optimal value table
    format_table(optimal_V, "\n--- Final Optimal Value Table (V*) ---")

    # Extract and display the optimal policy
    optimal_policy = extract_policy(optimal_V)
    format_table(optimal_policy, "\n--- Optimal Policy (π*) ---")

Convergence reached at iteration 11!

--- Final Optimal Value Table (V*) ---


,0,1,2,3,4,5,6,7,8,9
0,36.92,42.13,47.93,54.36,47.93,42.13,47.93,42.13,36.92,32.23
1,42.13,47.93,54.36,61.51,54.36,0.00,54.36,47.93,42.13,36.92
2,47.93,54.36,61.51,69.46,61.51,69.46,61.51,54.36,47.93,42.13
3,54.36,61.51,0.00,78.29,69.46,78.29,69.46,61.51,54.36,47.93
4,61.51,69.46,78.29,88.10,0.00,88.10,78.29,69.46,0.00,42.13
5,69.46,78.29,88.10,99.00,0.00,99.00,0.00,61.51,54.36,47.93
6,61.51,0.00,78.29,88.10,99.00,88.10,78.29,69.46,61.51,54.36
7,54.36,61.51,69.46,78.29,88.10,78.29,0.00,61.51,54.36,47.93
8,47.93,54.36,61.51,69.46,78.29,69.46,61.51,54.36,47.93,42.13
9,42.13,47.93,54.36,0.00,69.46,61.51,54.36,47.93,42.13,36.92



--- Optimal Policy (π*) ---


,0,1,2,3,4,5,6,7,8,9
0,↓,↓,↓,↓,↓,←,↓,↓,↓,↓
1,↓,↓,↓,↓,↓,💣,↓,↓,↓,↓
2,↓,↓,→,↓,↓,↓,↓,↓,↓,↓
3,↓,↓,💣,↓,←,↓,↓,↓,←,←
4,↓,↓,↓,↓,💣,↓,←,←,💣,↑
5,→,→,→,→,👑,←,💣,↑,↓,↓
6,↑,💣,↑,↑,↑,↑,←,←,←,←
7,↑,→,↑,↑,↑,↑,💣,↑,↑,↑
8,↑,↑,↑,↑,↑,↑,←,↑,↑,↑
9,↑,↑,↑,💣,↑,↑,↑,↑,↑,↑


# Q-Learning (With Epsilon Decay)

### Imports

In [ ]:
import numpy as np
import pandas as pd
import time
from IPython.display import display, clear_output

### Environment Definition and Setup



In [ ]:
ROWS = 10
COLS = 10
STATES = [(row, col) for row in range(ROWS) for col in range(COLS)]

"""
(0, 0), (0, 1), (0, 2),
(1, 0), (1, 1), (1, 2),
(2, 0), (2, 1), (2, 2)
"""

GOAL = (5, 4)
GOAL_REWARD = 100
PITS = {(3, 2), (4,8), (9, 3), (7, 6), (6, 1), (1, 5), (4, 4), (5, 6)}
PIT_REWARD = -50
TERMINAL_STATES = {GOAL}|PITS

STEP_REWARD = -1

# BONUSES = {(5, 5)}
# BONUS_RWARD = 10

ACTIONS = {
    'UP': (-1, 0),
    'DOWN': (1, 0),
    'LEFT': (0, -1),
    'RIGHT': (0, 1)
}

# R(x,x,s') - the immidiate reward for moving into s'=next_state
def get_reward(next_state):
    reward = 0
    if next_state == GOAL:
        reward = reward + GOAL_REWARD
    if next_state in PITS:
        reward = reward + PIT_REWARD
    # if next_state in BONUSES:
    #     reward = reward + BONUS_REWARD
    return reward + STEP_REWARD        # 1 point penalty for all moves


GAMMA = 0.9  # Discount factor
CONVERGENCE_THRESHOLD = 0.1  # When to stop iterating


### Helper Functions

In [ ]:
ACTION_ARROWS = {'UP': '↑', 'DOWN': '↓', 'LEFT': '←', 'RIGHT': '→'}  # For drawing the optimal policy grid later on

def get_next_state(state, action):  # Returns (s', R(s,a,s'))
    # If state is terminal it stays terminal
    if state in TERMINAL_STATES:
        return state, 0

    next_state = (state[0]+action[0], state[1]+action[1])
    # If new row or col is out of bounds stay in the same state
    if not (0 <= next_state[0] < ROWS and 0 <= next_state[1] < COLS):
        next_state = state

    return next_state, get_reward(next_state)


def display_policy_table(policy_matrix, title=""):

    col_names = [f'{c}' for c in range(COLS)]

    df = pd.DataFrame(policy_matrix, index=list(range(ROWS)), columns=col_names)

    if title:
        print(title)
    display(df)

def format_q_table(q_table):
  print("\n--- Q-Learning Final Q-Table (Grid View) ---")
  for action_name, action_vec in ACTIONS.items():
    print(f'\nQ(S, {action_name}):')
    q_grid_for_action = np.zeros((ROWS, COLS))
    for state in STATES:
      q_grid_for_action[state] = q_table[state][action_name]
    df_q_grid = pd.DataFrame(q_grid_for_action, index=list(range(ROWS)), columns=list(range(COLS)))
    display(df_q_grid.style.format("{:,.2f}"))

### Q-Learning Hyperparameters

In [ ]:
ALPHA = 0.1     # Learning Rate (how much we trust new information)
GAMMA = 0.9     # Discount Factor (value of future rewards)
EPSILON_START = 1.0 # Initial exploration rate
EPSILON_END = 0.01  # Minimum exploration rate
N_EPISODES = 20000 # Total episodes for training
DECAY_RATE = 0.0001 # Rate at which epsilon decays per step (approx 10,000 steps to reach END)

# Initialize Q-Table: A dictionary mapping state -> action -> Q-value
# Example: Q_table[(0, 0)] = {'UP': 0.0, 'DOWN': 0.0, 'LEFT': 0.0, 'RIGHT': 0.0}
Q_table = {s: {a: 0.0 for a in ACTIONS} for s in STATES}

### Epsilon-Greedy Policy Helper Function

In [ ]:

def choose_action(state, epsilon):
    # Check if the state is terminal
    if state in TERMINAL_STATES:
        return 'STOP'

    # Decide between Exploration (random) and Exploitation (best-known)
    if np.random.random() < epsilon:
        # EXPLORATION: Choose a random action
        return np.random.choice(list(ACTIONS.keys()))
    else:
        # EXPLOITATION: Choose the action with the highest Q-value
        q_values = Q_table[state]
        # Handle ties by randomly selecting one of the max Q-value actions
        max_q = max(q_values.values())
        best_actions = [a for a, q in q_values.items() if q == max_q]
        return np.random.choice(best_actions)

### Q-Learning Algorithm (Training Loop)

In [ ]:
def run_q_learning():
    epsilon = EPSILON_START

    # Track performance for plotting/monitoring
    rewards_per_episode = []

    print("Starting Q-Learning Training...")

    for episode in range(1, N_EPISODES + 1):
        # Start state (top left corner)
        current_state = (0, 0)
        episode_reward = 0

        while current_state not in TERMINAL_STATES:

            # Epsilon-Greedy Action Selection (Exploitation/Exploration)
            action = choose_action(current_state, epsilon)

            # Environment Interaction (Get S', R = Take step from S to S' and get immidiate reward R)
            next_state, reward = get_next_state(current_state, ACTIONS[action])
            episode_reward += reward

            # Find max_a' Q(s', a')
            if next_state in TERMINAL_STATES:
                max_future_q = 0 # No future from a terminal state
            else:
                max_future_q = max(Q_table[next_state].values())

            #### Q-Learning Update Rule (Bellman Equation) ####

            # Temporal Difference Target (New Info): R + gamma * max_a' Q(s', a')
            td_target = reward + GAMMA * max_future_q

            # Temporal Difference Error: (TD_target - Q(s, a)) = (New Info - Old Info)
            td_error = td_target - Q_table[current_state][action]

            # Update Q-Value: Q(s,a) += alpha * TD_error
            Q_table[current_state][action] += ALPHA * td_error

            ####################################################

            # Transition to next state
            current_state = next_state

            # Epsilon decay
            epsilon = max(EPSILON_END, epsilon - DECAY_RATE)

        rewards_per_episode.append(episode_reward)

        # Display progress every 2000 episodes
        if episode % 2000 == 0:
            clear_output(wait=True)
            print(f"Episode: {episode}/{N_EPISODES}. Epsilon: {epsilon:.4f}. Avg Reward (Last 100): {np.mean(rewards_per_episode[-100:]):.2f}")

    print("Training finished.")
    return Q_table

### Policy Extraction

In [ ]:
def extract_policy(Q_table):
  policy_matrix = np.full((ROWS, COLS), '', dtype=object)

  for state in STATES:
    if state == GOAL:
        policy_matrix[state[0], state[1]] = "👑"
        continue
    if state in PITS:
        policy_matrix[state[0], state[1]] = "💣"
        continue

    # Find the action with the maximum Q-value
    q_values = Q_table[state]

    # In case all states are 0 (unvisited states)
    if all(q == 0 for q in q_values.values()):
          policy_matrix[state[0], state[1]] = "❔"
          continue

    max_q = max(q_values.values())
    # Choose the action (a) that yields the best Q(s,a) for this state (s)
    best_action_name = [a for a, q in q_values.items() if q == max_q][0]
    policy_matrix[state[0], state[1]] = ACTION_ARROWS[best_action_name]

  return policy_matrix

### Execution

In [ ]:
if __name__ == "__main__":

    # Run the training
    final_Q_table = run_q_learning()

    # Extract the optimal policy
    optimal_policy = extract_policy(final_Q_table)

    # Display the result
    format_q_table(final_Q_table)
    display_policy_table(optimal_policy, "\n--- Q-Learning Optimal Policy ---")
    print(f"\n👑 = Goal (+{GOAL_REWARD}) | 💣 = Pit (-{PIT_REWARD}) | ❔ = Unvisited State")

Episode: 20000/20000. Epsilon: 0.0100. Avg Reward (Last 100): 89.38
Training finished.

--- Q-Learning Final Q-Table (Grid View) ---

Q(S, UP):


,0,1,2,3,4,5,6,7,8,9
0,32.21,36.91,12.50,-1.46,-1.56,-0.93,-0.35,-0.30,-0.20,0.00
1,16.85,36.90,21.24,1.42,-1.58,0.00,-0.22,-0.30,-0.10,-0.10
2,15.27,42.11,47.92,54.31,0.80,-9.69,-0.20,-0.38,-0.20,-0.19
3,12.56,3.58,0.00,61.51,12.29,-0.10,-0.11,-0.27,-0.27,-0.27
4,0.13,5.24,-33.22,69.41,0.00,-0.07,-0.10,-0.19,0.00,-0.19
5,-0.70,1.13,28.34,78.29,0.00,0.00,0.00,0.00,0.00,0.00
6,-0.47,0.00,27.66,87.98,26.83,0.00,-5.10,0.00,-0.27,0.00
7,-0.30,-5.10,1.81,0.21,0.79,0.00,0.00,-0.10,-0.10,0.00
8,-0.28,-0.10,-0.10,0.00,0.00,0.00,0.00,0.00,-0.10,0.00
9,-0.19,-0.10,-0.10,0.00,0.00,0.00,0.00,0.00,-0.10,0.00



Q(S, DOWN):


,0,1,2,3,4,5,6,7,8,9
0,36.87,42.13,47.92,25.12,0.85,-39.33,-0.34,-0.51,-0.20,0.00
1,23.59,47.93,54.36,61.51,-0.71,0.00,-0.11,-0.19,-0.38,-0.30
2,0.18,52.88,-51.00,69.46,61.48,0.31,-0.19,-0.27,-0.53,-0.37
3,6.21,61.11,0.00,78.29,-39.33,-0.19,-0.19,-0.19,-13.82,-0.34
4,-0.67,9.78,52.26,88.10,0.00,0.00,-5.10,-0.10,0.00,-0.10
5,-0.83,-31.24,5.26,77.50,0.00,0.00,0.00,-0.10,-0.19,-0.10
6,-0.47,0.00,-0.29,-0.10,-0.03,0.00,0.00,-0.10,0.00,-0.10
7,-0.21,-0.19,-0.10,-0.10,0.00,0.00,0.00,0.00,0.00,-0.10
8,-0.22,-0.10,-0.10,-5.10,0.00,0.00,0.00,0.00,0.00,-0.10
9,-0.37,-0.10,-0.10,0.00,0.00,0.00,0.00,0.00,-0.10,0.00



Q(S, LEFT):


...


Q(S, RIGHT):


,0,1,2,3,4,5,6,7,8,9
0,36.92,42.06,2.53,-1.92,-1.37,-0.76,-0.47,-0.36,-0.19,-0.19
1,42.13,47.91,35.35,2.83,-31.24,0.00,-0.27,-0.43,-0.28,-0.10
2,47.92,54.36,61.51,54.13,-0.78,-0.47,-0.27,-0.34,-0.43,-0.34
3,1.10,-47.70,0.00,61.45,8.93,-0.41,-0.28,-0.46,-0.36,-0.27
4,14.34,69.40,78.29,-50.99,0.00,-0.19,-0.10,0.00,0.00,-0.19
5,8.80,49.13,88.09,99.00,0.00,0.00,0.00,-0.10,-0.10,-0.10
6,-17.54,0.00,5.74,2.95,0.00,0.00,-0.10,-0.10,0.00,0.00
7,-0.34,-0.35,-0.27,-0.10,0.00,0.00,0.00,0.00,0.00,0.00
8,-0.10,-0.11,-0.19,-0.10,0.00,0.00,0.00,0.00,0.00,0.00
9,-0.27,-0.27,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.27



--- Q-Learning Optimal Policy ---


,0,1,2,3,4,5,6,7,8,9
0,→,↓,↓,↓,↓,→,↓,←,→,↑
1,→,↓,↓,↓,←,💣,↓,↓,↑,↑
2,→,→,→,↓,↓,↓,←,←,↑,↑
3,↑,↓,💣,↓,←,←,←,↓,↑,↑
4,→,→,→,↓,💣,↓,↑,←,💣,↓
5,→,→,→,→,👑,❔,💣,↑,↑,↑
6,↓,💣,↑,↑,↑,❔,↓,↑,↓,↑
7,↓,←,↑,↑,↑,❔,💣,↓,↓,↑
8,←,↑,↑,↑,↑,❔,❔,❔,↓,↑
9,↑,↑,→,💣,❔,❔,❔,❔,←,↑



👑 = Goal (+100) | 💣 = Pit (--50) | ❔ = Unvisited State
